In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import os
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry


In [2]:
# Checkpoint file names
PRODUCT_URLS_FILE = 'product_urls_checkpoint.csv'
REVIEWS_FILE = 'reviews_checkpoint.csv'

In [3]:
# Function to create a session with retry logic
def create_session():
    session = requests.Session()
    retry = Retry(
        total=5,
        read=5,
        connect=5,
        backoff_factor=0.3,
        status_forcelist=(500, 502, 503, 504),
        allowed_methods=["HEAD", "GET", "OPTIONS"]
    )
    adapter = HTTPAdapter(max_retries=retry)
    session.mount("http://", adapter)
    session.mount("https://", adapter)
    return session

In [4]:
# Function to get product URLs from a single listing page
def get_product_urls(listing_url):
    session = create_session()
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}
    try:
        response = session.get(listing_url, headers=headers)
        response.raise_for_status()
    except requests.exceptions.RequestException as e:
        print(f"Request failed: {e}")
        return []

    soup = BeautifulSoup(response.content, 'html.parser')
    product_urls = []
    
    for a_tag in soup.find_all('a', class_='core'):
        url = a_tag.get('href')
        if url:
            product_urls.append('https://www.jumia.com.ng' + url)
            
    return product_urls

In [5]:
# Function to get the maximum number of review pages for a product
def get_max_review_pages(product_url):
    session = create_session()
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}
    try:
        response = session.get(product_url, headers=headers)
        response.raise_for_status()
    except requests.exceptions.RequestException as e:
        print(f"Request failed: {e}")
        return 1

    soup = BeautifulSoup(response.content, 'html.parser')
    pagination = soup.find('div', class_='pg-w -ptm -pbxl')
    if pagination:
        pages = pagination.find_all('a')
        page_numbers = [int(page.text.strip()) for page in pages if page.text.strip().isdigit()]
        if page_numbers:
            return max(page_numbers)
    return 1  # Default to 1 if pagination is not found


In [6]:
# Function to get product reviews from a single page
def get_product_reviews(url):
    session = create_session()
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}
    try:
        response = session.get(url, headers=headers)
        response.raise_for_status()
    except requests.exceptions.RequestException as e:
        print(f"Request failed: {e}")
        return []

    soup = BeautifulSoup(response.content, 'html.parser')
    
    reviews = []
    review_elements = soup.find_all('article', class_='-pvs -hr _bet')

    for element in review_elements:
        review = {}
        
        rating_tag = element.find('div', class_='stars _m _al -mvs')
        review['rating'] = rating_tag.text.split(' out of ')[0].strip() if rating_tag else 'N/A'
        
        title_tag = element.find('h3', class_='-m -fs16 -pvs')
        review['title'] = title_tag.text.strip() if title_tag else 'N/A'
        
        body_tag = element.find('p', class_='-pvs')
        review['body'] = body_tag.text.strip() if body_tag else 'N/A'
        
        date_tag = element.find('span', class_='-prs')
        review['date'] = date_tag.text.strip() if date_tag else 'N/A'
        
        name_tag = element.find('span', string=lambda x: x and x.startswith('by '))
        review['name'] = name_tag.text.replace('by ', '').strip() if name_tag else 'N/A'
        
        reviews.append(review)

    return reviews

In [7]:
# Function to get all reviews for a given product URL
def get_all_reviews(product_url):
    max_pages = get_max_review_pages(product_url)
    all_reviews = []
    for page in range(1, max_pages + 1):
        url = f"{product_url}?page={page}"
        reviews = get_product_reviews(url)
        all_reviews.extend(reviews)
        time.sleep(0.5)  # Add a delay to avoid hitting the server too quickly
    
    return all_reviews

In [8]:
# Function to save product URLs to a CSV file (checkpoint)
def save_product_urls_checkpoint(product_urls):
    df = pd.DataFrame(product_urls, columns=['product_url'])
    df.to_csv(PRODUCT_URLS_FILE, index=False)

In [9]:
# Function to load product URLs from a CSV file (checkpoint)
def load_product_urls_checkpoint():
    if os.path.exists(PRODUCT_URLS_FILE):
        df = pd.read_csv(PRODUCT_URLS_FILE)
        return df['product_url'].tolist()
    return []

In [10]:
# Function to save reviews to a CSV file (checkpoint)
def save_reviews_checkpoint(reviews):
    df = pd.DataFrame(reviews)
    if os.path.exists(REVIEWS_FILE):
        df.to_csv(REVIEWS_FILE, mode='a', header=False, index=False)
    else:
        df.to_csv(REVIEWS_FILE, index=False)

In [11]:
# Function to load reviews from a CSV file (checkpoint)
def load_reviews_checkpoint():
    if os.path.exists(REVIEWS_FILE):
        return pd.read_csv(REVIEWS_FILE).to_dict('records')
    return []

In [12]:
# Function to scrape multiple products
def scrape_multiple_products(product_urls):
    all_product_reviews = load_reviews_checkpoint()
    
    # Get already scraped product URLs to avoid reprocessing
    scraped_product_urls = {review['product_url'] for review in all_product_reviews}
    
    for product_url in product_urls:
        if product_url not in scraped_product_urls:
            print(f"Scraping reviews for: {product_url}")
            product_reviews = get_all_reviews(product_url)
            for review in product_reviews:
                review['product_url'] = product_url  # Add product URL to each review for checkpointing
            all_product_reviews.extend(product_reviews)
            save_reviews_checkpoint(product_reviews)  # Save progress
            scraped_product_urls.add(product_url)
    
    return all_product_reviews

In [13]:
def main():
    base_listing_url = 'https://www.jumia.com.ng/catalog/?q=gown&page='  # Base URL for the listing pages
    total_listing_pages = 50  # Total number of listing pages to scrape

    # Load previously scraped product URLs to avoid reprocessing
    all_product_urls = load_product_urls_checkpoint()
    last_page_scraped = len(all_product_urls) // 40  # Assuming 40 products per page

    # Step 1: Get product URLs from all listing pages
    for page in range(last_page_scraped + 1, total_listing_pages + 1):
        listing_url = f"{base_listing_url}{page}"
        product_urls = get_product_urls(listing_url)
        all_product_urls.extend(product_urls)
        print(f"Page {page}: Found {len(product_urls)} product URLs.")
        save_product_urls_checkpoint(all_product_urls)  # Save progress
    
    print(f"Total product URLs found: {len(all_product_urls)}")

    # Step 2: Scrape reviews for each product URL
    all_reviews = scrape_multiple_products(all_product_urls)

    if all_reviews:
        df = pd.DataFrame(all_reviews)
        # Reorder the columns
        df = df[['name', 'rating', 'date', 'title', 'body', 'product_url']]
        print(df)
        # Save to a CSV file
        filename = os.path.join(os.getcwd(), 'jumia_NLP_dataset.csv')
        df.to_csv(filename, index=False)
        print(f"File saved as {filename}")

        # Additional confirmation
        if os.path.exists(filename):
            print(f"Confirmed: {filename} has been saved.")
        else:
            print(f"Error: {filename} was not saved.")
    else:
        print("No reviews found.")

if __name__ == '__main__':
    main()

Total product URLs found: 2000
Scraping reviews for: https://www.jumia.com.ng/ladies-corporate-gown-fashion-mpg4061222.html
Scraping reviews for: https://www.jumia.com.ng/fashion-ladies-single-gown-188942876.html
Scraping reviews for: https://www.jumia.com.ng/fashion-ladies-charming-gown-193524735.html
Scraping reviews for: https://www.jumia.com.ng/fashion-ladies-cun-gown-130364377.html
Scraping reviews for: https://www.jumia.com.ng/generic-very-fluffy-bath-robeswimming-pool-side-robe-98292394.html
Scraping reviews for: https://www.jumia.com.ng/fashion-ladies-glass-gown-157848127.html
Scraping reviews for: https://www.jumia.com.ng/ladies-valor-gown-fashion-mpg3012493.html
Scraping reviews for: https://www.jumia.com.ng/generic-wooden-rabbit-wall-hanger-280312054.html
Scraping reviews for: https://www.jumia.com.ng/fashion-ladies-clean-gown-193507553.html
Scraping reviews for: https://www.jumia.com.ng/fashion-ladies-beautiful-corporate-gown-311676361.html
Scraping reviews for: https://www

Scraping reviews for: https://www.jumia.com.ng/generic-4-pcs-set-jm066-arrow-shaped-sticky-wall-seamless-hook-4-pcs-set-white-268202515.html
Scraping reviews for: https://www.jumia.com.ng/stainless-steel-cylinder-hanger-bathroom-non-perforated-storage-clothes-hook-size14mm-black-generic-mpg2373511.html
Scraping reviews for: https://www.jumia.com.ng/generic-10pcs-powerful-non-mark-storage-hooks-hangers-fixtures-295281571.html
Scraping reviews for: https://www.jumia.com.ng/generic-3-pcs-removable-office-desk-side-hook-253461749.html
Scraping reviews for: https://www.jumia.com.ng/generic-3-pcs-nail-free-silicone-seamless-hook-253461832.html
Scraping reviews for: https://www.jumia.com.ng/generic-2pcs-household-wall-hanging-basketball-storage-display-stand-229553998.html
Scraping reviews for: https://www.jumia.com.ng/generic-a352a-for-blink-camera-routers-wall-bracket-229553924.html
Scraping reviews for: https://www.jumia.com.ng/generic-2-pcs-cartoon-hanger-storage-rack-wall-mounted-rack-25

Scraping reviews for: https://www.jumia.com.ng/generic-10-pcs-nonpunching-adhesive-thumb-hook-cable-organizer-319650018.html
Scraping reviews for: https://www.jumia.com.ng/generic-10-pcs-non-punching-adhesive-thumb-hook-cable-organizer-319653476.html
Scraping reviews for: https://www.jumia.com.ng/generic-304-stainless-steel-nail-free-mop-rack4-clip-5-hook-green-327241652.html
Scraping reviews for: https://www.jumia.com.ng/generic-3-pcs-office-deskside-removable-bag-hook-333563473.html
Scraping reviews for: https://www.jumia.com.ng/generic-2pcs-tow-shackles-trailer-hook-for-110-traxxas-hsp-redcat-333580818.html
Scraping reviews for: https://www.jumia.com.ng/generic-3-pcs-office-deskside-removable-bag-hook-333583821.html
Scraping reviews for: https://www.jumia.com.ng/generic-wall-mounted-mop-holder-333573988.html
Scraping reviews for: https://www.jumia.com.ng/generic-wall-hooksbathroom-kitchen-self-adhesive-hooks3-pcs-347907833.html
Scraping reviews for: https://www.jumia.com.ng/generic-

Scraping reviews for: https://www.jumia.com.ng/generic-4-pcs-set-jm066-arrow-shaped-sticky-wall-seamless-hook-4-pcs-set-orange-320340657.html
Scraping reviews for: https://www.jumia.com.ng/generic-3-pcs-nail-free-silicone-seamless-hook-320828041.html
Scraping reviews for: https://www.jumia.com.ng/generic-3-pcs-removable-office-desk-side-hook-318842292.html
Scraping reviews for: https://www.jumia.com.ng/generic-square-wall-hanging-shelf-flower-pot-319620842.html
Scraping reviews for: https://www.jumia.com.ng/generic-2-pcs-cartoon-hanger-storage-rack-wall-mounted-rack-318842416.html
Scraping reviews for: https://www.jumia.com.ng/generic-stainless-steel-door-rear-coat-hook-specification-6-hooks-318852161.html
Scraping reviews for: https://www.jumia.com.ng/generic-3-pcs-whale-shape-shelf-household-wall-mounted-storage-box-318843829.html
Scraping reviews for: https://www.jumia.com.ng/generic-multi-layer-pull-out-drain-storage-rack-black-1-layer-pump-318854181.html
Scraping reviews for: http

Scraping reviews for: https://www.jumia.com.ng/generic-y01-head-14-inches-wardrobe-hanging-rod-clothes-rail-338840909.html
Scraping reviews for: https://www.jumia.com.ng/generic-10pcs-powerful-nonmark-storage-hooks-hangers-fixtures-330483392.html
Scraping reviews for: https://www.jumia.com.ng/generic-2pcs-stainless-steel-kitchen-cabinet-draw-over-door-hook-clothes-hanger-holdersilver-338839030.html
Scraping reviews for: https://www.jumia.com.ng/generic-y01b-plastic-head-14-inches-wardrobe-hanging-rod-clothes-rail-338840649.html
Scraping reviews for: https://www.jumia.com.ng/generic-dog-bathrobe-towel-dog-drying-coat-microfiber-fast-drying-super-absorbent-pet-dog-cat-bath-robe-towelm-green-360904085.html
Scraping reviews for: https://www.jumia.com.ng/generic-2-pcs-deskside-schoolbag-hook-338841284.html
Scraping reviews for: https://www.jumia.com.ng/generic-male-hand-display-stand-for-jewelry-glove-left-hand-white-330491710.html
Scraping reviews for: https://www.jumia.com.ng/generic-2-pc

...

Scraping reviews for: https://www.jumia.com.ng/fashion-ladies-social-gown-216676182.html
Scraping reviews for: https://www.jumia.com.ng/fashion-ladies-renew-gown-216665652.html
Scraping reviews for: https://www.jumia.com.ng/ladies-bliss-gown-fashion-mpg3011474.html
Scraping reviews for: https://www.jumia.com.ng/fashion-ladies-juice-gown-115618528.html
Scraping reviews for: https://www.jumia.com.ng/fashion-ladies-smile-gown-115617268.html
Scraping reviews for: https://www.jumia.com.ng/fashion-red-nightgowns-backless-women-sleep-wear-night-gowns-with-thong-sets-v-neck-lace-young-girl-stripe-design-black-white-red-maa-236150349.html
Scraping reviews for: https://www.jumia.com.ng/fashion-sexy-lace-lingerie-robe-sheer-long-dress-evening-nightgown-nightie-sleepwear-newest-stripper-outfit-dancewear-exotic-dresses-white-stockings-229093067.html
Scraping reviews for: https://www.jumia.com.ng/fashion-classy-and-elegant-long-gown-for-ladies.-269382678.html
Scraping reviews for: https://www.jumia.

Scraping reviews for: https://www.jumia.com.ng/fashion-unique-and-classy-ladies-fashionable-gown-255489837.html
Scraping reviews for: https://www.jumia.com.ng/fashion-adorable-and-smart-ladies-corporate-gownno-broche-276904331.html
Scraping reviews for: https://www.jumia.com.ng/fashion-blue-ladies-fancy-gown-258389755.html
Scraping reviews for: https://www.jumia.com.ng/fashion-sexy-ladies-beautiful-night-gown-354929549.html
Scraping reviews for: https://www.jumia.com.ng/elegant-cooperate-gown-for-ladies-fashion-mpg3605491.html
Scraping reviews for: https://www.jumia.com.ng/fashion-box-pleat-blue-classy-gown-261579410.html
Scraping reviews for: https://www.jumia.com.ng/fashion-long-elegant-ladies-gown-267795837.html
Scraping reviews for: https://www.jumia.com.ng/fashion-ladies-sexy-night-gown-263292758.html
Scraping reviews for: https://www.jumia.com.ng/fashion-classic-cooperate-office-ladies-gown-286428254.html
Scraping reviews for: https://www.jumia.com.ng/fashion-beautiful-sexy-corpo

Scraping reviews for: https://www.jumia.com.ng/fashion-beautiful-and-classy-ladies-gown-267783974.html
Scraping reviews for: https://www.jumia.com.ng/fashion-adorable-and-classy-ladies-fashionable-gown-237766839.html
Scraping reviews for: https://www.jumia.com.ng/fashion-gown-short-elegant-dress-94810542.html
Scraping reviews for: https://www.jumia.com.ng/generic-corporate-gown-for-ladies-318180306.html
Scraping reviews for: https://www.jumia.com.ng/fashion-beautiful-gown-for-fine-ladies-311676510.html
Scraping reviews for: https://www.jumia.com.ng/fashion-women-elegant-casual-mini-gown-white-214439209.html
Scraping reviews for: https://www.jumia.com.ng/fashion-beautiful-trendy-casualcooperate-flare-gown-322287039.html
Scraping reviews for: https://www.jumia.com.ng/generic-classy-layered-girls-jeans-gown-292539169.html
Scraping reviews for: https://www.jumia.com.ng/fashion-ladies-beautiful-corporate-office-gown-324136732.html
Scraping reviews for: https://www.jumia.com.ng/fashion-class

Scraping reviews for: https://www.jumia.com.ng/fashion-classic-cooperate-office-ladies-gown-286428042.html
Scraping reviews for: https://www.jumia.com.ng/fashion-ladies-sexy-gown-without-accessories-357881214.html
Scraping reviews for: https://www.jumia.com.ng/fashion-classy-and-stylish-ladies-fashionable-gown-263052764.html
Scraping reviews for: https://www.jumia.com.ng/fashion-classic-cooperate-office-ladies-gown-253203801.html
Scraping reviews for: https://www.jumia.com.ng/fashion-beautiful-quality-abaya-ladies-gown-256614588.html
Scraping reviews for: https://www.jumia.com.ng/fashion-maxi-free-abaya-gown-for-beautiful-ladies-259231145.html
Scraping reviews for: https://www.jumia.com.ng/fashion-formal-pinafore-gown-redwhite-255545938.html
Scraping reviews for: https://www.jumia.com.ng/fashion-corporate-elegant-trendy-ladies-gown-309807743.html
Scraping reviews for: https://www.jumia.com.ng/fashion-unique-corporate-gown-for-ladies-ladies-292604492.html
Scraping reviews for: https://w

Scraping reviews for: https://www.jumia.com.ng/fashion-admirable-and-lovely-ladies-gown-278170828.html
Scraping reviews for: https://www.jumia.com.ng/fashion-ladies-night-wear-gown-308908638.html
Scraping reviews for: https://www.jumia.com.ng/fashion-maxi-orange-gown-abaya-229238447.html
Scraping reviews for: https://www.jumia.com.ng/fashion-trendy-and-stylish-ladies-gown-290439608.html
Scraping reviews for: https://www.jumia.com.ng/fashion-three-quarter-royal-blue-flare-short-gown-202794778.html
Scraping reviews for: https://www.jumia.com.ng/fashion-long-elegant-ladies-gown-248456172.html
Scraping reviews for: https://www.jumia.com.ng/fashion-trendy-beautiful-stylish-gown-black-and-animal-skin-134331879.html
Scraping reviews for: https://www.jumia.com.ng/fashion-beautiful-multicolor-strip-fancy-ladies-gown-268022272.html
Scraping reviews for: https://www.jumia.com.ng/fashion-ladies-beautiful-night-gown-203551353.html
Scraping reviews for: https://www.jumia.com.ng/fashion-a-lady-abayah